**Image2Image Pipeline for Stable Diffusion using Gradio**

In [1]:
!nvidia-smi

Fri Dec  1 12:29:40 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           Off | 00000000:62:00.0 Off |                    0 |
| N/A   49C    P0              55W / 300W |  10237MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os

# Get your own process ID (PID)
pid = os.getpid()

print(f"Your process ID (PID): {pid}")

Your process ID (PID): 3194835


In [3]:
# !pip install -qq -U transformers==4.26.1 ftfy gradio diffusers

In [4]:
# !pip install -qq "ipywidgets>=7,<8"

In [5]:
# !pip install diffusers

In [6]:
# !pip install gradio

In [7]:
# !pip install 'transformers>=4.15.0,<=4.26.1'

In [8]:
# !pip freeze > requirements.txt

In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [10]:
import sys
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

from diffusers import StableDiffusionImg2ImgPipeline

import asyncio

import os
clip_interrogator_path = '/home/ochirpurev/.local/lib/python3.10/site-packages/'
sys.path.append(clip_interrogator_path)

from clip_interrogator import Config, Interrogator

2023-12-01 12:29:42.342413: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 12:29:43.373293: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

from diffusers import DiffusionPipeline

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16",
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)
pipe = pipe

vae/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

**Import Gradio framework and required dependencies**

In [12]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from PIL import Image as PILImage

**Define a folder to save the sketches**

In [13]:
save_folder = "/home/ochirpurev/sketches"  # Use an accessible directory in Colab
!mkdir -p $save_folder

**Define a prompt here**

In [14]:
prompt="a stydio ghibli-like art style a house in a countryside in a cartoon like world and with children book illustration and beautiful anime-like"

**Enhance Enhancer function**

In [15]:
def enhance_image_sketch(sketch, prompt_input):
    # Generate a unique filename
    filename = f"{save_folder}/sketch.png"
    image_data = np.array(sketch)
    # extent = (0, sketch.width, 0, sketch.height)

    # Save the sketch as a PNG file
    plt.imsave(filename, image_data, cmap="viridis", origin="upper")
    
    # Open the saved image using PIL
    saved_image = PILImage.open(filename)
    saved_image = saved_image.convert("RGB").resize((512, 512)) 
    generator = torch.Generator(device=device).manual_seed(1024)
    # return_image
    with autocast("cuda"):
        return_image = pipe(prompt=prompt_input, image=saved_image, strength=0.65, guidance_scale=7.5, generator=generator).images[0]

    return return_image

**Run the sketch_enhancer to draw the input sketch**

In [16]:
input_sketchpad =gr.Paint(label="Draw Here", brush_radius=4, type="pil", shape=(512, 512))
# sketch_enhancer = gr.Interface(fn=save_as_pil_image, inputs=input_sketchpad, outputs='image').queue().launch(share=True, inline=False)
sketch_enhancer = gr.Interface(
    fn=enhance_image_sketch, 
    inputs=input_sketchpad, 
    outputs='image', 
    live=True).queue().launch(share=True, inline=False)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://2dac8dbd0765360715.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [17]:
config = Config(clip_model_name="ViT-L-14/openai")
ci_vitl = Interrogator(config)
ci = ci_vitl

Loading BLIP model...
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
Loading CLIP model...
Loaded CLIP model and data in 10.47 seconds.


In [18]:
import openai 
openai.api_key = ""
messages = [ {"role": "system", "content":  
              "You are a intelligent assistant. That is helping to create a children book story, sentence by sentence"} ] 

In [19]:
def generate_next_prompt(generated_image):
    image_content = 'Then, she decided to look at the mountains'
    # image = Image.fromarray(generated_image)
    # image = image.convert('RGB')
    # image_content2 = ci.interrogate_classic(image)
    
    # Generate a unique filename
    filename = f"{save_folder}/sketch.png"
    image_data = np.array(generated_image)
    plt.imsave(filename, image_data, cmap="viridis", origin="upper")
    # Open the saved image using PIL
    saved_image = PILImage.open(filename)
    saved_image = saved_image.convert("RGB").resize((512, 512))
    image_content2 = ci.interrogate_classic(saved_image)
    
    if(type(image_content2) == str):
        image_content=image_content2
    
    instruction= "Create one sentence to continue a children story, based on a picture that containts the following information: "
    messages.append({"role": "user", "content": instruction + image_content},) 
    chat = openai.ChatCompletion.create( 
            model="gpt-3.5-turbo", messages=messages 
        ) 
    reply = chat.choices[0].message.content     
    return reply

In [20]:
with gr.Blocks() as demo:
    with gr.Row():
        text_Prompt = gr.Textbox(label="Sketch The following Story :)", interactive = False, value = 'Anna was walking in a forest')
    with gr.Row():
        with gr.Column(scale=2, min_width=600):
            input_sketchpad =gr.Paint(label="Draw Here", brush_radius=4, type="pil", shape=(512, 512))
        with gr.Column(scale=2, min_width=600):
            output_Image = gr.Image(interactive = False)
    with gr.Row():
        btn_generate = gr.Button("Generate")
        btn_continue = gr.Button("Next Chapter")
        
    btn_generate.click(enhance_image_sketch, inputs=[input_sketchpad, text_Prompt], outputs=output_Image)
    btn_continue.click(generate_next_prompt, inputs=output_Image, outputs=text_Prompt)
    
demo.launch(share=True, inline=False)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://a5823ae31781a4213f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
